In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os
sys.path.append('/content/drive/MyDrive/ColabNotebooks/DL/ViT')
os.chdir("/content/drive/MyDrive/ColabNotebooks/DL/ViT")


# for sorting out images
# os.chdir("/content/drive/MyDrive/ColabNotebooks/DL/ViT/dataset/images")

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=3a9a5be9b019dec75052007bc966671a9bb83732ea3fa8dc356b12566da4a734
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import locale
print(locale.getpreferredencoding())

import locale
def getpreferredencoding(do_setlocale = True):
 return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install pillow
!pip install torchvision

UTF-8


In [ ]:
import torch
import matplotlib.pyplot as plt
from data_util import get_data_from_kaggle, restructure_date_information, get_static_df, one_label_scale_static_df, scale_stock_data
from constant import Constant
import pandas as pd
from torch.utils.data import DataLoader ,random_split
from data_util import get_feature_length
import glob
import data
from data import ViT_Dataset
import sklearn
from transformers import ViTConfig, ViTModel
import torch.nn as nn
import torch.nn.functional as F
from ViT import ViT_Model

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cuda


In [ ]:
constant = Constant() # initialize variables we will use



---

\

Preprocesssing

\

---

In [ ]:
market = 'sp500' # [forbes2000, nasdaq, nyse, sp500]
year = '2017'

In [ ]:
# stock = get_data_from_kaggle(market = 'sp500', start_date = f'01-01-{year}')

In [ ]:
# generating static df can take a while because there is a rate limit with yfinance api
# in my case I don't need statick
# static = get_static_df(stock,  constant.static_variables)
# stock.to_csv(f'dataset/sp500_{year}.csv', index=False)
stock = pd.read_csv(f'dataset/sp500_{year}.csv')

In [ ]:
# static = get_static_df(stock,  constant.static_variables)

In [ ]:
stock = stock.dropna()

In [ ]:
# note: stock name and order might not be the same as the others as the names are out of order
# check with team after

In [ ]:
stock_scaled, stock_scalar = scale_stock_data(stock, constant.columns_to_scale)
stock_scaled

,Date,Low,Open,Volume,High,Close,Adjusted Close,Stock Name
0,2017-01-03,0.429773,0.360857,0.035805,0.295354,0.375313,0.177820,O
1,2017-01-04,0.440909,0.358690,0.032017,0.313003,0.396845,0.195154,O
2,2017-01-05,0.450000,0.370968,0.042778,0.361536,0.436154,0.226799,O
3,2017-01-06,0.481591,0.406596,0.033126,0.361277,0.434151,0.225187,O
4,2017-01-09,0.481591,0.398411,0.035000,0.352193,0.423135,0.216319,O
...,...,...,...,...,...,...,...,...
605210,2022-12-06,0.492183,0.487677,0.292503,0.467268,0.480273,0.550906,CME
605211,2022-12-07,0.459954,0.474656,0.286528,0.456749,0.451745,0.525353,CME
605212,2022-12-08,0.455509,0.441992,0.196124,0.437273,0.460546,0.539887,CME
605213,2022-12-09,0.474179,0.459648,0.290996,0.447793,0.472499,0.550656,CME


In [ ]:
processed_df = restructure_date_information(stock_scaled)

In [ ]:
processed_df.head()

,Low,Open,Volume,High,Close,Adjusted Close,Stock Name,Month,Day,Day of The Week,Week of The Year
0,0.429773,0.360857,0.035805,0.295354,0.375313,0.177820,O,0,2,1,0
1,0.440909,0.358690,0.032017,0.313003,0.396845,0.195154,O,0,3,2,0
2,0.450000,0.370968,0.042778,0.361536,0.436154,0.226799,O,0,4,3,0
3,0.481591,0.406596,0.033126,0.361277,0.434151,0.225187,O,0,5,4,0
4,0.481591,0.398411,0.035000,0.352193,0.423135,0.216319,O,0,8,0,1


In [ ]:
# static now

In [ ]:
# static.to_csv(f'dataset/sp500_{year}_static.csv')
static = pd.read_csv(f"dataset/sp500_{year}_static.csv", index_col=0)
static_df = one_label_scale_static_df(static,constant.static_variables)



---

\

---



---



In [ ]:
history_cat_feature_num_list, history_cont_feature_num = get_feature_length(processed_df, constant.feature_variables)
static_cat_feature_num_list , static_cont_feature_num  = get_feature_length(static, constant.static_variables)
future_cat_feature_num_list , _                        = get_feature_length(processed_df, constant.future_feature)
_                           , prediction_con           = get_feature_length(processed_df, constant.prediction_feature)

In [ ]:
history_cat_feature_num_list, history_cont_feature_num

([12, 31, 5, 53], 5)

In [ ]:
history_length = 90
prediction_length = 15

In [ ]:
# use this if want to reset ViT Dataset
# del stock_ViT_dataset

In [ ]:
stock_ViT_dataset = ViT_Dataset(stock_df= processed_df ,
                        static_df=static_df,
                        constant_variable=constant,
                        history_length= history_length,
                        prediction_length= prediction_length,
                        device = device)

In [ ]:
# counter = 0
# for i in range(len(stock_ViT_dataset)):
  # stock_ViT_dataset[i]
  # counter += 1
  # if counter % 100000 == 0:
  #   print(counter)

# print(counter)



---

Training



---



In [ ]:
batch_size = 128

In [ ]:
train_size = int(0.65 * stock_ViT_dataset.__len__())
test_size = len(stock_ViT_dataset) - train_size

In [ ]:
train_dataset, test_dataset = random_split(stock_ViT_dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True)

In [ ]:
count =0
for static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction, image_tensor, prediction_classification in train_loader:
    break
print(image_tensor.device)
print(image_tensor.size())

cuda:0
torch.Size([128, 3, 64, 64])


In [ ]:
 # training

import torch.optim as optim
# Call the model with the patches_tensor
model = ViT_Model(image_size = 64, patch_size=16, embedding_dim=256, num_heads=8, num_classes=2, num_layers=10, mlp_dim = 512, channels = 3, embedding_dropout = 0.0 )
model.to(device)
# loss function
loss_fn = nn.CrossEntropyLoss().to(device)  # binary cross-entropy loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # optimizer


In [ ]:
model.train()
num_epochs = 7
counter = 0
min_loss = float('inf')
# training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):

      # this is where I will use patches_tensor and prediction_classification from each batch
      static_cont_input, static_cat_input, history_cont_input, history_cat_input, future_input, prediction, patches_tensor, prediction_classification = batch
      # forward pass
      outputs = model(patches_tensor)  # (batch_size, 1)
      loss = loss_fn(outputs, prediction_classification) # loss

      if batch_idx % 400 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")


      # backprop
      optimizer.zero_grad()
      loss.backward()
      # gradient clipping to help convergence?
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # clip max_norm to 1.0

      optimizer.step()

      running_loss += loss.item()

    if running_loss < min_loss:
      torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ViT/ViT_model_{year}.pth')
      min_loss = running_loss
    # avg epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

Epoch [1/7], Batch [1/2856], Loss: 0.6922
Epoch [1/7], Batch [401/2856], Loss: 0.7171
Epoch [1/7], Batch [801/2856], Loss: 0.6993
Epoch [1/7], Batch [1201/2856], Loss: 0.6656
Epoch [1/7], Batch [1601/2856], Loss: 0.6878
Epoch [1/7], Batch [2001/2856], Loss: 0.7032


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ViT/ViT_model_{year}.pth')



---
Prediction


---




In [ ]:
# model prediction

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Get a sample from the dataset
    sample = ViT_Dataset[0]  # Assuming this gets one sample

    # Extract the input patches
    patches_tensor = sample[6].unsqueeze(0)  # Add batch dimension

    # Get the prediction (probability of class 1)
    prediction_prob = model(patches_tensor)

    # Convert probability to class (0 or 1)
    predicted_class = (prediction_prob > 0.5).int()

    print("Predicted Class:", predicted_class.item())  # Output: 0 or 1





---

\

Creating Model Instance

\

---

In [ ]:
# pretraining?

In [ ]:
hidden_size = 128
batch_size = 128
dropout = .2
num_head = 4
lr= 0.0001
momentum=0.9
num_epochs = 10
max_grad_norm = 1
split_ratio = 0.8

In [ ]:
model = ViT(
            device = device)
model.to(device)

ViT model extracts features